In [6]:
import requests
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from operator import contains

### Load environment variables

In [7]:
load_dotenv()

api_key = os.getenv('API_KEY')

### Get artist's mbid

In [8]:
# api-endpoint
URL = "https://api.setlist.fm/rest/1.0/search/artists"
  
artist_name = "Rozalén"
  
# defining a params dict for the parameters to be sent to the API
PARAMS = {'artistName': artist_name}
  
# sending get request and saving the response as response object
r = requests.get(url = URL, params = PARAMS, headers={"Accept": "application/json", "Accept-Language": "es",  "x-api-key": api_key})

print(r.text)

{
   "type" : "artists",
   "itemsPerPage" : 30,
   "page" : 1,
   "total" : 3,
   "artist" : [ {
      "mbid" : "71296827-461f-4a9e-a02d-dcc3c293486b",
      "name" : "Santiago Cruz & Rozalén",
      "sortName" : "Cruz, Santiago & Rozalén",
      "url" : "https://www.setlist.fm/setlists/santiago-cruz-and-rozalen-3bf60884.html"
   }, {
      "mbid" : "0432dc38-a03b-4325-9609-1caada06f47a",
      "name" : "Rozalén",
      "sortName" : "Rozalén",
      "disambiguation" : "",
      "url" : "https://www.setlist.fm/setlists/rozalen-6bdc5eb2.html"
   }, {
      "mbid" : "14cd3589-be97-4bfa-8c30-93a14208a947",
      "name" : "Rozalén & Ana Belén",
      "sortName" : "Rozalén & Belén, Ana",
      "url" : "https://www.setlist.fm/setlists/rozalen-and-ana-belen-1bf705dc.html"
   } ]
}


### Get artist's setlists:

In [9]:
# Manually, we get the MBID:
mbid = "0432dc38-a03b-4325-9609-1caada06f47a"
# api-endpoint
URL = f"https://api.setlist.fm/rest/1.0/artist/{mbid}/setlists"

# sending get request and saving the response as response object
r = requests.get(url = URL, headers={"Accept": "application/json", "Accept-Language": "es",  "x-api-key": api_key})

data = r.json()
print(data.keys())

dict_keys(['type', 'itemsPerPage', 'page', 'total', 'setlist'])


#### Formatting and predict next setlist:

In [10]:
setlists = data["setlist"]
reduced_sets = {}
total_songs_sung = []

for idx, i in enumerate(setlists):
    if i['sets']['set'] != []:
        print(f"Date: {i['eventDate']}")
        # print(f"Sets: {i['sets']}")
        print(f"URL: {i['url']}")
        reduced_sets[idx] = {'eventDate': i['eventDate'], 'url': i['url'], 'sets': i['sets']}
        # Amount of songs:
        total_songs_sung.append(len(i['sets']['set'][0]['song']))

        

Date: 24-09-2022
URL: https://www.setlist.fm/setlist/rozalen/2022/complejo-deportivo-cantarranas-madrid-spain-33b04839.html
Date: 17-09-2022
URL: https://www.setlist.fm/setlist/rozalen/2022/pistas-de-atletismo-fuente-de-la-nina-guadalajara-spain-63b00213.html
Date: 18-08-2022
URL: https://www.setlist.fm/setlist/rozalen/2022/plaza-de-toros-herrera-del-duque-spain-73b32e59.html
Date: 02-08-2022
URL: https://www.setlist.fm/setlist/rozalen/2022/jardines-terramar-sitges-spain-4bb3b37a.html
Date: 11-06-2022
URL: https://www.setlist.fm/setlist/rozalen/2022/auditorio-de-la-alameda-jaen-spain-63b3a237.html


In [11]:
total = len(reduced_sets)
# Get the average amount of songs in a concert
lenght = int(np.floor(np.mean(total_songs_sung)))
print(f"Our predicted setlist will consider the {lenght} top frequent songs.")

Our predicted setlist will consider the 19 top frequent songs.


In [12]:
all_songs = []

for idx, i in enumerate(reduced_sets.values()):
    list_names = i['sets']['set'][0]['song']
    for j in list_names:
        # This part is most likely exclusively for Rozalén and her encores:
        if "/" in j['name']:
            # print(j['name'])
            aux = j["name"].split("/")
            aux = [all_songs.append(x.strip()) for x in aux]
            # all_songs.append(aux)
        else:
            all_songs.append(j['name'])

In [13]:
df = pd.DataFrame(all_songs, columns=['Songs'])
freq = df.groupby(by=['Songs']).agg({'Songs': 'count'}).rename(columns={'Songs':'Frequency'}).sort_values(by=['Frequency'], ascending=False)
# Select the n top most frequent songs sung:
print(f"This is a simple predicted setlist for {artist_name}'s concert:")
str_size = max([len(x) for x in all_songs])
line = pd.DataFrame({"Frequency": "--"}, index=["-"*str_size])
freq2 = pd.concat([freq.iloc[:lenght], line, freq.iloc[lenght:]])
freq2

This is a simple predicted setlist for Rozalén's concert:


,Frequency
80 veces,5
Saltan chispas,5
Vivir,5
Vuelves,5
La puerta violeta,5
A tu vida,5
La Línea,5
Justo,5
Girasoles,5
Este tren,5
